In [1]:
!pip freeze | grep "^\(scikit-learn\|pandas\|numpy\|torch\)=="

numpy==1.21.6
pandas==1.3.5
scikit-learn==1.0.2


# Домашнее задание о разметке последовательностей.



**Задача**: научиться автоматически выделять именованные сущности в художественной литературе.

Подзадачи:
* [в тетради приведён код] подготовить данные для обучения sequence labeling моделей
* [в тетради приведён код] обучить, оценить бейзлайновую модель NER
* реализовать модель, превосходящую по метрикам бейзлайновую модель.


### получение данных

Будем использовать набор с корпусом LitBank. Корпус собран из популярных художественных произведений на английском языке и сожержит разметку по именованным сущностям и событиям. Объем корпуса таков: 100 текстов по примерно 2000 слов каждый. 

Корпус описан в статьях:
* David Bamman, Sejal Popat, Sheng Shen, An Annotated Dataset of Literary Entities http://people.ischool.berkeley.edu/~dbamman/pubs/pdf/naacl2019_literary_entities.pdf
* Matthew Sims, Jong Ho Park, David Bamman, Literary Event Detection,  http://people.ischool.berkeley.edu/~dbamman/pubs/pdf/acl2019_literary_events.pdf

Корпус доступен в репозитории проекта:  https://github.com/dbamman/litbank

Структура корпуса в репозитории такова. 
Первый уровень: 
* entities &mdash; разметка по сущностям.
В корпусе используются 6 типов именованных сущностей: PER, LOC, ORG, FAC, GPE, VEH (имена, локации, организации, помещения, топонимы, средства перемещния), допускаются вложенные сущности. 
* events &mdash; разметка по событиям

Второй уровень:
* brat &mdash; рабочие файлы инструмента разметки brat, ann-файлы содержат разметку, txt-файлы – сырые тексты 
* tsv &mdash; tsv-файлы содержат разметку в IOB формате,

In [2]:
!git clone https://github.com/dbamman/litbank

!TRAIN_DATA_FPATH="train.bio" && TEST_DATA_FPATH="test.bio" \
 && rm -f $TRAIN_DATA_FPATH $TEST_DATA_FPATH \
 && FILES=$(ls -1 litbank/entities/tsv/*) && N_FILES=$(echo "$FILES" | wc -l) \
 && N_TRAIN_FILES=$((N_FILES * 0,8)) && N_TEST_FILES=$((N_FILES * 0,2)) \
 && TRAIN_FILES=$(echo "$FILES" | head -n $N_TRAIN_FILES) \
 && TEST_FILES=$(echo "$FILES" | tail -n $N_TEST_FILES) \
 && for fn in "$TRAIN_FILES"; do cut -d $'\t' -f1,3 $fn >> $TRAIN_DATA_FPATH; done \
 && for fn in "$TEST_FILES"; do cut -d $'\t' -f1,3 $fn >> $TEST_DATA_FPATH; done

TRAIN_DATA_FPATH = "train.bio"
TEST_DATA_FPATH = "test.bio"

!wc -l {TRAIN_DATA_FPATH} {TEST_DATA_FPATH}
!head -n 3 {TRAIN_DATA_FPATH} {TEST_DATA_FPATH} | column -s $'\t' -t

Cloning into 'litbank'...
remote: Enumerating objects: 1183, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 1183 (delta 15), reused 103 (delta 5), pack-reused 1056
Receiving objects: 100% (1183/1183), 40.71 MiB | 19.11 MiB/s, done.
Resolving deltas: 100% (132/132), done.
 18049 train.bio
  4744 test.bio
 22793 total
==> train.bio <==
CHAPTER            O
I                  O
In                 O
==> test.bio <==
CHAPTER            O
I                  O
Mr                 B-PER


Посмотрим, как выглядит корпус

In [3]:
print("==> train data part <==")
!grep -m 1 -B 10 -A 10 I-P {TRAIN_DATA_FPATH} 

print("\n\n==> test data part <==")
!grep -m 1 -B 10 -A 10 I-[^P] {TEST_DATA_FPATH} 

==> train data part <==
.	O

Michaelmas	O
term	O
lately	O
over	O
,	O
and	O
the	O
Lord	B-PER
Chancellor	I-PER
sitting	O
in	O
Lincoln	O
's	O
Inn	O
Hall	O
.	O

Implacable	O
November	O


==> test data part <==
marking	O
ink	O
,	O
retail	O
value	O
sixpence	O
(	O
price	O
in	O
Verloc	B-FAC
’s	I-FAC
shop	I-FAC
one-and-sixpence	O
)	O
,	O
which	O
,	O
once	O
outside	O
,	O
he	O


Данные &mdash; набор текстов английской литературы. 
* тексты поделены на предложения, предложения отделены друг от друга пустой строкой
* тексты токенизированы 
  * каждый токен находится на очередной строке
  * каждому токену в соответствие поставлен тег в **BIO-нотации**.

О **BIO-нотации** можно прочитать [тут](https://habr.com/ru/company/abbyy/blog/449514/).



Наша модель должна уметь обработать предложение: сопоставить каждому токену предложения некоторую метку.

Для упрощения дальнейшей работы давайте сразу считаем набор данных в переменные питона.
Удобно иметь тренировочную запись, соответствующую каждому предложению, отделённой от записей, соответствующих другим предложениям.

Организуем хранение наборов данных так: для набора данных будем иметь два списка -- список-про-тексты и список-про-теги. 
Каждый элемент такого списка &mdash; тоже список: список токенов или соответствующих тегов отдельного предложения. 

In [4]:
def read_bio_dataset(dataset_fpath):
    sentences_tokens, sentences_tags = [], []
    curr_sent_tokens, curr_sent_tags = [], []

    for line in open(dataset_fpath, encoding="utf-8"):
        line = line.strip()
        if not line:
            sentences_tokens.append(curr_sent_tokens)
            curr_sent_tokens = []
            sentences_tags.append(curr_sent_tags)
            curr_sent_tags = []
        else:
            curr_token, curr_tag = line.split('\t')
            curr_sent_tags.append(curr_tag)
            curr_sent_tokens.append(curr_token)
            
    sentences_tokens.append(curr_sent_tokens)
    sentences_tags.append(curr_sent_tags)

    return sentences_tokens, sentences_tags


train_sents_tokens, train_sents_tags = read_bio_dataset(TRAIN_DATA_FPATH)
test_sents_tokens, test_sents_tags = read_bio_dataset(TEST_DATA_FPATH)

print(f"e.g. train sentences tokens:\t{str(train_sents_tokens[10:12])}")
print(f"e.g. train sentences tags:\t{str(train_sents_tags[10:12])}")
print()
print(f"e.g. test sentences tokens:\t{str(test_sents_tokens[10:12])}")
print(f"e.g. test sentences tags:\t{str(test_sents_tags[10:12])}")

e.g. train sentences tokens:	[['Fog', 'on', 'the', 'Essex', 'marshes', ',', 'fog', 'on', 'the', 'Kentish', 'heights', '.'], ['Fog', 'creeping', 'into', 'the', 'cabooses', 'of', 'collier-brigs', ';', 'fog', 'lying', 'out', 'on', 'the', 'yards', 'and', 'hovering', 'in', 'the', 'rigging', 'of', 'great', 'ships', ';', 'fog', 'drooping', 'on', 'the', 'gunwales', 'of', 'barges', 'and', 'small', 'boats', '.']]
e.g. train sentences tags:	[['O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]

e.g. test sentences tokens:	[['These', 'customers', 'were', 'either', 'very', 'young', 'men', ',', 'who', 'hung', 'about', 'the', 'window', 'for', 'a', 'time', 'before', 'slipping', 'in', 'suddenly', ';', 'or', 'men', 'of', 'a', 'more', 'mature', 'age', ',', 'but', 'looking', 'generally', 'as', 'if', 'they', 'were', 'not

Соберём вспомогательные переменные -- `vocab` (словарь) и `tagset` (набор тегов).
Они нам потом пригодятся для того, чтобы представлять слова и теги при помощи чисел.

Также чуть позже мы воспользуемся специальными токенами -- *паддингами* -- 
и при обработке текстовых цепочек, и при обработке цепочек тегов.   
Сейчас заблаговременно запомним и эти паддинги.

In [5]:
PAD_TOKEN = "PAD_TOKEN"
vocab = set(token 
            for sentence_tokens in train_sents_tokens+test_sents_tokens
            for token in sentence_tokens)
vocab.add(PAD_TOKEN)
vocab = list(vocab)

PAD_TAG = "PAD_TAG"
tagset = set(tag
            for sentence_tags in train_sents_tags+test_sents_tags
            for tag in sentence_tags)
tagset.add(PAD_TAG)
tagset = list(tagset)


print(f"vocab size: {str(len(vocab))}, tagset_size: {str(len(tagset))}")
print("vocab samples:", vocab[:5])
print("tagset samples:", tagset[:5])

vocab size: 4357, tagset_size: 8
vocab samples: ['1810', 'boasted', 'bright', 'sought', 'Ghost']
tagset samples: ['I-FAC', 'I-LOC', 'B-PER', 'O', 'B-LOC']


In [6]:
sorted(tagset, 
       key=lambda x: ('-' in x,
                      x.split('-', maxsplit=1)[-1], 
                      x.split('-', maxsplit=1)[0]),
       reverse=False)
# хитрая сортировка чтобы веселее жилось. 
# хочется посмотреть на то, какие есть теги, но так, чтобы они были по порядку.

['O', 'PAD_TAG', 'B-FAC', 'I-FAC', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER']

Переменные `vocab` (словарь) и `tagset` (набор тегов) нам потом пригодятся для того, чтобы представлять слова и теги при помощи чисел. Давайте для удобства заранее заведём отображения элементов словаря\тагсета на числа и обратно.


In [7]:
ix2word = dict(enumerate(vocab))
word2ix = {w:ix for ix, w in ix2word.items()}

ix2tag = dict(enumerate(tagset))
tag2ix = {t:ix for ix, t in ix2tag.items()}

print("word2ix samples:", list(word2ix.items())[:5])
print("tag2ix samples:", list(tag2ix.items())[:5])

word2ix samples: [('1810', 0), ('boasted', 1), ('bright', 2), ('sought', 3), ('Ghost', 4)]
tag2ix samples: [('I-FAC', 0), ('I-LOC', 1), ('B-PER', 2), ('O', 3), ('B-LOC', 4)]


Теперь мы умеем превращать токены в числа.  
Превратим обучающие последовательности (тексты и последовательности тегов) в последовательности чисел.

In [8]:
train_sents_digitized = [[word2ix[tok] for tok in sent_toks]
                         for sent_toks in train_sents_tokens]

test_sents_digitized = [[word2ix[tok] for tok in sent_toks]
                         for sent_toks in test_sents_tokens]


train_sent_tags_digitized = [[tag2ix[tag] for tag in sent_tags]
                             for sent_tags in train_sents_tags]
test_sent_tags_digitized = [[tag2ix[tag] for tag in sent_tags]
                            for sent_tags in test_sents_tags]

print(train_sents_digitized[10])
print(train_sent_tags_digitized[10])
print()
print(test_sents_digitized[10])
print(test_sent_tags_digitized[10])

[2129, 4134, 1144, 1477, 3868, 2448, 3322, 4134, 1144, 3616, 3606, 4237]
[3, 3, 3, 4, 1, 3, 3, 3, 3, 3, 3, 3]

[3191, 2108, 3081, 2203, 1916, 1092, 2201, 2448, 1600, 4354, 736, 1144, 2084, 701, 3422, 1115, 1586, 2645, 3468, 1833, 4218, 3825, 2201, 756, 3422, 64, 2499, 1138, 2448, 3889, 4138, 645, 2812, 9, 3923, 3081, 248, 3468, 4046, 4237]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


Посмотрим, как теперь выглядит какой-то случайный обучающий пример.

In [9]:
train_data = list(zip(train_sents_digitized, train_sent_tags_digitized))
test_data = list(zip(test_sents_digitized, test_sent_tags_digitized))

print(train_data[10])
print()
print(test_data[10])

([2129, 4134, 1144, 1477, 3868, 2448, 3322, 4134, 1144, 3616, 3606, 4237], [3, 3, 3, 4, 1, 3, 3, 3, 3, 3, 3, 3])

([3191, 2108, 3081, 2203, 1916, 1092, 2201, 2448, 1600, 4354, 736, 1144, 2084, 701, 3422, 1115, 1586, 2645, 3468, 1833, 4218, 3825, 2201, 756, 3422, 64, 2499, 1138, 2448, 3889, 4138, 645, 2812, 9, 3923, 3081, 248, 3468, 4046, 4237], [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])


Теперь это набор чисел, представляющих текст предложения и набор чисел, представляющих теги токенов этого текста. 

##### О train, test, val подвыборках (subsamples)

При обучении нейронной сети каждую эпоху происходит 
1. итерация обучения (forward+backward modes)
2. итерация оценки (no backward)

После обучения мы захотим оценить её на отложенных данных. 
Отложенный набор данных &mdash; это наш *test*, а ещё давайте откусим часть *train* для промежуточных оценок.
Назовём эту часть данных *val*.

Ещё на всякий случай перемешаем примеры из набора данных. 
`train_test_split` делает это по умолчанию, `shuffle` из `sklearn` тоже умеет перемешивать коллекции.  

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


train_data, val_data = train_test_split(train_data, test_size=0.2)
test_data = shuffle(test_data)


Опять на всякий случай посмотрим на случайные примеры из *train*, *val*, *test*.


In [11]:
print(train_data[10], 
      [ix2word[w_ix] for w_ix in train_data[10][0]], 
      [ix2tag[t_ix] for t_ix in train_data[10][1]],
      sep='\n', end='\n***\n')

print(val_data[10], 
      [ix2word[w_ix] for w_ix in val_data[10][0]], 
      [ix2tag[t_ix] for t_ix in val_data[10][1]],
      sep='\n', end='\n***\n')

print(test_data[10], 
      [ix2word[w_ix] for w_ix in test_data[10][0]], 
      [ix2tag[t_ix] for t_ix in test_data[10][1]],
      sep='\n', end='\n***\n')

([2896, 2724, 3710, 3422, 3922, 1173, 2448, 1760, 3945, 3022, 2724, 1587, 924, 3922, 2052, 4237], [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
['There', 'would', 'be', 'a', 'new', 'Ayah', ',', 'and', 'perhaps', 'she', 'would', 'know', 'some', 'new', 'stories', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
***
([3620, 349, 2172, 2448, 3017, 148, 2448, 2675, 2724, 4139, 4181, 3858, 2847, 2246, 4047, 2448, 1831, 2675, 1706, 248, 285, 1916, 3260, 3835, 2378, 2007, 4237], [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
['For', 'one', 'daughter', ',', 'his', 'eldest', ',', 'he', 'would', 'really', 'have', 'given', 'up', 'any', 'thing', ',', 'which', 'he', 'had', 'not', 'been', 'very', 'much', 'tempted', 'to', 'do', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
***
([3976, 3081, 2100, 3468, 1144, 762, 46, 1760, 3415, 281

## наивный бейзлайн

Предположим, что если мы видим какое-то слово, то с ним сразу всё понятно.
Например, про слово *Michael* понятно, что это имя.

Давайте запомним, с каким тегом чаще всего встретилось каждое слово набора данных, и, встретив незнакомое предложение, протеггируем его следующим образом: встретив токен, припишем ему самый частый встретившийся рядом с этим токеном тег.

Для каждого токена из обучающей выборки найдём самый частый соответствующий ему тег.

In [12]:
from collections import defaultdict

token2tag2freq = defaultdict(lambda: defaultdict(int))

for train_sentence in train_data:
    tokens_ixes, tags_ixes = train_sentence[0], train_sentence[1]
    tokens = [ix2word[tok_ix] for tok_ix in tokens_ixes]
    tags = [ix2tag[tag_ix] for tag_ix in tags_ixes]
    for token, tag in zip(tokens, tags):
        token2tag2freq[token][tag] += 1

token2most_frequent_tag = dict()
for token, tag2freq in token2tag2freq.items():
    token2most_frequent_tag[token] = max(tag2freq, key=lambda tag: tag2freq[tag])


print("TOKEN", "MOST FREQUENT TAG", sep='\t')
for token, most_frequent_tag in list(token2most_frequent_tag.items())[:5]:
    print(token, most_frequent_tag, sep='\t')
print("...", "...", sep='\t')

for token, most_frequent_tag in [(tok, tag) 
                                 for tok, tag in token2most_frequent_tag.items()
                                 if tag != "O"][:5]:
    print(token, most_frequent_tag, sep='\t')
print("...", "...", sep='\t')

TOKEN	MOST FREQUENT TAG
Horses	O
,	O
scarcely	O
better	O
;	O
...	...
Walter	I-PER
Esq.	I-PER
larger	I-FAC
rat-hole	I-FAC
John	I-PER
...	...


Воспользуемся извлечённой информацией для теггирования отложенной тестовой выборки

In [13]:
test_sentences_predictions = []
for test_sentence in test_data:
    tokens_ixes, tags_ixes = test_sentence[0], test_sentence[1]
    tokens = [ix2word[tok_ix] for tok_ix in tokens_ixes]
    tags = [ix2tag[tag_ix] for tag_ix in tags_ixes]

    previous_tag = ''
    predicted_tags = []
    for token in tokens:
        predicted_tag = token2most_frequent_tag.get(token, 'O')  # О для незнакомых слов
        predicted_tags.append(predicted_tag)
        previous_tag = predicted_tag
    
    predicted_tags_ixes = [tag2ix[tag] for tag in predicted_tags]
    sentence_prediction = (tokens_ixes, predicted_tags_ixes)

    test_sentences_predictions.append(sentence_prediction)

In [14]:
print(test_sentences_predictions[0], 
      [ix2word[w_ix] for w_ix in test_sentences_predictions[0][0]], 
      [ix2tag[t_ix] for t_ix in test_sentences_predictions[0][1]],
      sep='\n', end='\n***\n')

([571, 3236, 2423, 2697, 4134, 1208, 3413, 3702, 348, 3995, 3422, 4091, 1760, 691, 2147, 4237], [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
['It', 'was', 'only', 'later', 'on', 'that', 'Winnie', 'obtained', 'from', 'him', 'a', 'misty', 'and', 'confused', 'confession', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
***


### оценка решения

Теперь, когда мы получили наивное решение, оценим её на тестовой выборке.

Для наглядности, запомним предсказания модели, а также будем использовать две метрики оценки: среднее по примерам среднее качество на примере и среднее по примерам бинарное качество на примере.

Среднее (`mean`) качество теггирования на входном тексте пропорционально количеству правильно теггированых токенов текста.
Бинарное (`joint`) качество теггирования на входном тексте более строго: оно равно единице когда все токены теггированы правильно и равно нулю в всех остальных случаях. Таким образом, мы смотрим, правильно ли модель обработала пример или нет.


In [15]:
import numpy as np

def binary_sentence_accuracy(sentence_predicted_tags, sentence_true_tags):
    return int(sentence_predicted_tags == sentence_true_tags)

def mean_sentence_accuracy(sentence_predicted_tags, sentence_true_tags):
    sentence_len = len(sentence_predicted_tags)
    equal_elems_num = sum(1 
                          for tag_pred, tag_gold in zip(sentence_predicted_tags, 
                                                        sentence_true_tags)
                          if tag_pred == tag_gold)
    return equal_elems_num/sentence_len if sentence_len else 0

def metrics(sentences_predictions, sentences_truths):
    mean_accuracies = []
    binary_accuracies = []
    for sentence_predictions, sentence_truths in zip(sentences_predictions, sentences_truths):
        assert sentence_predictions[0] == sentence_truths[0]
        sentence_tokens = sentence_predictions[0]

        sentence_tags_predicted_ixes = sentence_predictions[1]
        sentence_tags_predicted = [ix2tag[tag_ix] for tag_ix in sentence_tags_predicted_ixes]

        sentence_tags_true_ixes = sentence_truths[1]
        sentence_tags_true = [ix2tag[tag_ix] for tag_ix in sentence_tags_true_ixes]

        curr_sentence_mean_acc = mean_sentence_accuracy(sentence_tags_predicted, 
                                                        sentence_tags_true)
        
        curr_sentence_binary_acc = binary_sentence_accuracy(sentence_tags_predicted, 
                                                            sentence_tags_true)
        
        mean_accuracies.append(curr_sentence_mean_acc)
        binary_accuracies.append(curr_sentence_binary_acc)
    
    mean_mean_accuracy = np.mean(mean_accuracies)
    joint_accuracy = np.mean(binary_accuracies)

    return {"mean": mean_mean_accuracy, "joint": joint_accuracy}

Посмотрим на метрики нашего решения

In [16]:
metrics(test_data, test_sentences_predictions)

{'joint': 0.6144578313253012, 'mean': 0.9384193626120357}

Скорее всего, лучше всего наша модель справляется с проставлением метки `O` всем словам в предложении: скорее всего для каждого слова метка `O` &mdash; самая частотная. В таком случае, joint метрику должны в первую очередь повышать случаи, когда в предложении на самом деле нет именованных сущностей. Модель предскажет метку `O` всем словам, и это окажется верным ответом.

## нейросетевой бейзлайн

Давайте теперь реализуем нейросетевой бейзлайн для нашей задачи. Его нужно будет превзойти, сдавая задание.

### группировка данных в батчи

Современные вычислительные системы (нам особенно релевантны видеокарты) хороши тем, что способны за один момент времени совершить несколько подобных операций. Например, за один момент времени можно *(одновременно, параллельно)* посчитать значение одного и того же уравнения при нескольких разных входных переменных.

Это используется для ускорения обучения нейронных сетей: при обучении приходится 
1. вычислить результаты работы нейронной сети для нескольких входных примеров
2. по этим результатам оценить ошибку
3. понимая ошибку, обновить нужные параметры нейронной сети.  

Пункт *1.* кажется подходящим для описанного выше параллеллизма.


Чтобы воспользоваться этим параллелизмом, **соберём наши обучающие примеры в группы - *батчи***, вычисления для каждого батча впоследствии будут происходить распараллеленно.

In [17]:
def generate_batched_dataset(dataset, batch_size):

    batched_dataset = []
    curr_batch = []
    for data_entry_idx, (src, tgt) in enumerate(dataset, 1):
        curr_batch.append((src, tgt))
        if data_entry_idx % batch_size == 0:
            batched_dataset.append(list(zip(*curr_batch)))
            curr_batch = []
    batched_dataset.append(list(zip(*curr_batch)))

    return batched_dataset

BATCH_SIZE = 64

batched_train_data = generate_batched_dataset(train_data, BATCH_SIZE)
batched_val_data = generate_batched_dataset(val_data, BATCH_SIZE)
batched_test_data = generate_batched_dataset(test_data, BATCH_SIZE)


Посмотрим на какой-то из полученных батчей. Это два набора: набор цифрованных предложений и набор соответствующих этим предложениям цепочек тегов (теги тоже цифрованы). 

Для проверки заметим, что длина каждого *i*-того по счёту в батче предложения совпадает с длиной соответствующей ему *i*-той по счёту в батче цепочки тегов. 

In [18]:
print(batched_test_data[-2])
print(list(map(len, batched_test_data[-2][0])))
print(list(map(len, batched_test_data[-2][1])))

[([120, 4137, 1942, 2910, 1144, 53, 589, 756, 396, 4237], [571, 1534, 3710, 1208, 120, 4137, 3236, 3445, 2378, 1569, 4209, 4237], [571, 1942, 3422, 3221, 756, 1144, 3041, 2448, 905, 1760, 1471, 2448, 1768, 2863, 3422, 1647, 3333, 3453, 1697, 3422, 1273, 1760, 1144, 4096, 756, 3422, 1938, 4237], [2555, 2556, 2675, 2849, 3468, 3864, 2177, 1208, 2675, 1706, 1933, 3422, 20, 3468, 3017, 732, 4237], [493, 3558, 1134, 1341, 3915, 3468, 1144, 2590, 3361, 756, 3558, 1405, 2448, 3923, 4216, 3468, 4130, 2448, 349, 3247, 113, 2448, 2812, 9, 1364, 2378, 764, 1144, 1427, 4230, 4237], [487, 2448, 2675, 3939, 1706, 2246, 125, 1381, 1831, 3236, 3891, 2779, 4218, 1760, 1586, 1144, 3452, 4114, 756, 1474, 4134, 1144, 3269, 756, 3017, 874, 2675, 3312, 3907, 2448, 3468, 3017, 3314, 1629, 710, 2448, 652, 701, 3676, 2518, 1144, 1651, 3829, 756, 3017, 2785, 3413, 4237], [2791, 2049, 3236, 64, 3867, 1715, 4237], [56, 2084, 1303, 183, 756, 64, 3825, 1737, 2036, 133, 850, 4218, 1231, 2742, 3468, 2597, 3206, 3914,

Стоит помнить, что в каждом батче у нас набор последовательностей, для которых мы будем проводить вычисления. Батч &mdash; это набор подобных друг другу элементов, благодаря подобности которых становится возможным одновременно вычислить что-то для каждого из элементов батча.

В нашем батче все **последовательности имеют разную длину**. 
Это нарушает идею о подобности элементов батча. 
Давайте элементы в каждом батче уподобим по длине, добавив к каждой последовательности специальный "хвост" из элементов &mdash; символов паддинга:

```
[[a, b, c], 
 [d, e], 
 [f]] 

превращается в 

[[a,   b,   c], 
 [d,   e, PAD], 
 [f, PAD, PAD]] 
```

In [19]:
def pad_batch_of_sequences(batch_sequences, padding_elem):
    longest_sequence_len = max([len(seq) for seq in batch_sequences])
    padded_sequence_len = longest_sequence_len #+ #1  # [последо, ватель, .., ность, (PAD, PAD,..)]
    padded_sequences = [seq + [padding_elem]*(padded_sequence_len - len(seq))
                        for seq in batch_sequences]
    
    return(padded_sequences)

def pad_batched_dataset_of_sequences_pairs(dataset_of_sequences_pairs,
                                           src_seq_pad_elem,
                                           tgt_seq_pad_elem):
    padded_dataset = [(pad_batch_of_sequences(src_seq, src_seq_pad_elem),
                       pad_batch_of_sequences(tgt_seq, tgt_seq_pad_elem))
                      for src_seq, tgt_seq in dataset_of_sequences_pairs]
    return padded_dataset


padded_batched_train_data = pad_batched_dataset_of_sequences_pairs(batched_train_data, 
                                                                   word2ix[PAD_TOKEN],
                                                                   tag2ix[PAD_TAG])

padded_batched_val_data = pad_batched_dataset_of_sequences_pairs(batched_val_data, 
                                                                 word2ix[PAD_TOKEN],
                                                                 tag2ix[PAD_TAG])

padded_batched_test_data = pad_batched_dataset_of_sequences_pairs(batched_test_data, 
                                                                  word2ix[PAD_TOKEN],
                                                                  tag2ix[PAD_TAG]) 


### описание нейронной сети и сопутствующего

In [20]:
import torch

In [21]:
# в переменной DEVICE запомнится полезная информация о видеокарте в нашей системе
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

Тренировка нейросети на наборе данных и оценка нейросети на наборе данных похожи. Единственное различие -- при тренировке мы используем оценку ошибки для обновления весов нейросети.

В pytorch если мы не планируем в ближайшие запуски обновлять веса нейросети, стоит перевести её в положение `eval`. Если планируем &mdash; в положение `test`.

In [22]:
import numpy as np

def batch_preds_aggregated_loss(pred_tgt, gold_tgt, criterion):

    batch_size = gold_tgt.shape[0]
    seq_len = gold_tgt.shape[1]

    batch_elems_losses = torch.mean(
        torch.stack([criterion(pred_tgt.view(batch_size, seq_len, -1)[batch_entry_idx],
                               gold_tgt.view(batch_size, seq_len)[batch_entry_idx])
                             for batch_entry_idx in range(batch_size)]))
    aggregated_loss = batch_elems_losses.mean()
    return aggregated_loss

def epoch_iter(model, batched_dataset, optimizer, criterion, update_weights):
    if update_weights:
        model.train()
    else:
        model.eval()
    
    iter_losses = []

    for src, tgt in batched_dataset:
        src_tensor = torch.tensor(src).to(DEVICE)
        tgt_tensor = torch.tensor(tgt).to(DEVICE)

        output, _ = model(src_tensor)

        batch_loss = batch_preds_aggregated_loss(output, tgt_tensor, criterion)
        
        if update_weights:
            batch_loss.backward()
            optimizer.step()

        iter_losses.append(batch_loss.item())
    
    return np.mean(iter_losses)


def train_epoch(model, batched_dataset, optimizer, criterion):
    return epoch_iter(model, batched_dataset, optimizer, criterion, update_weights=True)

def test_epoch(model, batched_dataset, optimizer, criterion):
    return epoch_iter(model, batched_dataset, optimizer, criterion, update_weights=False)

### запуск обучения

Опишем простую нейронную сеть на фреймворке pytorch.

Для теггирования последовательностей наивным бейзлайном можно считать однослойную LSTM-сеть.

Элементы последовательности попадают в LSTM-сеть через выучиваемый линейный слой (эмбеддингов). 

Выход LSTM-сети отображается на множество тегов ещё одним линейным слоем.

In [23]:
class LSTMModel(torch.nn.Module):
    def __init__(self, 
                 input_dim, output_dim,
                 emb_dim, hidden_dim):
        super().__init__()

        self.emb = torch.nn.Embedding(input_dim, emb_dim)
        self.lstm = torch.nn.LSTM(emb_dim, hidden_dim, bidirectional=True)
        self.fc = torch.nn.Linear(2*hidden_dim, output_dim)
    
    def forward(self, in_seqs_batch):
        embedded = self.emb(in_seqs_batch)
        lstm_out, hc = self.lstm(embedded)
        out = self.fc(lstm_out)
        return out, hc

Архитектура нашей сети такова: на вход модели подаётся вектор, он линейным слоем эмбеддингов отображается в некоторый другой вектор-эмбеддинг, эмбеддинг подаётся в LSTM-слой, выход LSTM-слоя отображается в некоторый вектор, являющийся выходным вектором всей нейросети.

Мы можем конфигурировать размерность векторов эмбеддингов и размерность слоя LSTM. Входные и выходные векторы &mdash; это наши слова и теги соответственно, и их размерности предопределены и равны размеру словаря и тагсета.

При работе с паддингами, можно помнить, что занятая паддингами часть цепочки токенов -- это наше техническое дополнение, не имеющее смысловой интерпретации в мире исходной задачи. Не стоит штрафовать модель за ошибки в работе с паддингами, такие ошибки всё равно находятся за границей осмысленной части текста, будем их игнорировать.

Стоит также помнить, что иногда в вычислениях мы опираемся на техническую переменную `device`, помогающую pytorch понять, есть ли шанс воспользоваться видеокартой в вычислениях.

# LSTMModel

Теперь, когда мы обучили модель, проверим работает ли она: оценим её на тестовой выборке.

Для наглядности, запомним предсказания модели, а также будем использовать две метрики оценки: среднее по примерам среднее качество на примере и среднее по примерам бинарное качество на примере.

Среднее (`mean`) качество теггирования на входном тексте пропорционально количеству правильно теггированых токенов текста.
Бинарное (`joint`) качество теггирования на входном тексте более строго: оно равно единице когда все токены теггированы правильно и равно нулю в всех остальных случаях. Таким образом, мы смотрим, правильно ли модель обработала пример или нет.


In [24]:
def batch_score(src_tensor, tgt_tensor):
    """returns mean and joint scores on data batch"""
    src = src_tensor.tolist()
    tgt = tgt_tensor.tolist()

    pred, _ = model(src_tensor)  # получаем предсказания модели на элементах батча
    
    batch_tok2tag2pred = []
    batch_joint_accuracies = []
    batch_mean_accuracies = []
    for some_src, some_tgt, tags_predicted_digitized in zip(src, 
                                                            tgt, 
                                                            pred.argmax(dim=-1).tolist()):
        # восстанавливаем токены, предсказанные и ожидаемые метки токенов, из их численных записей
        tokens = list(map(ix2word.get, some_src))
        tags_true = list(map(ix2tag.get, some_tgt))
        tags_pred = list(map(ix2tag.get, tags_predicted_digitized))

        tok2tag2pred = list(zip(tokens, tags_true, tags_pred))
        
        batch_tok2tag2pred.append(tok2tag2pred)

        correct_preds = []  # чтобы хранить верные предсказания модели в человекочитаемом виде
        actual_preds = []  # чтобы хранить *все* предсказания модели в человекочитаемом виде
        for tok, gold_tag, pred_tag in tok2tag2pred:
            if gold_tag == PAD_TAG:
                # нас не волнуют ошибки на паддингах, тк они находятся за пределами предложения
                break

            batch_entry_w_prediction = (tok, gold_tag, pred_tag)
            if gold_tag == pred_tag:
                correct_preds.append(batch_entry_w_prediction)
            actual_preds.append(batch_entry_w_prediction)

        correct_preds_num = len(correct_preds)
        joint_accuracy = int(correct_preds_num == len(actual_preds))  # 1 если не было ошибок, иначе 0
        mean_accuracy = correct_preds_num/len(actual_preds) if actual_preds else 0  # доля верных ответов
        batch_joint_accuracies.append(joint_accuracy)
        batch_mean_accuracies.append(mean_accuracy)


    return batch_tok2tag2pred, batch_joint_accuracies, batch_mean_accuracies


Будем учить модель несколько эпох. На каждой эпохе будем обращать внимание на ошибку на валидационной выборке: именно она информативно характеризует, обучилась ли модель.

Если валидационная ошибка не будет уменьшаться на протяжении длительного времени, остановим обучение модели, тк оно уже не приводит к наблюдаемым улучшениям.

In [25]:
'''
model = LSTMModel(len(word2ix), len(tag2ix), emb_dim=100, hidden_dim=100)

criterion = torch.nn.CrossEntropyLoss(ignore_index=tag2ix[PAD_TAG])
model = model.to(DEVICE)
optimizer = torch.optim.Adam(params=model.parameters())
criterion = criterion.to(DEVICE)


EPOCHS = 100
IMPATIENCE_LIMIT = 5

val_losses = [float("inf")] * (IMPATIENCE_LIMIT + 1)
for epoch_num in range(EPOCHS):
    train_loss = train_epoch(model, padded_batched_train_data, optimizer, criterion)
    val_loss = test_epoch(model, padded_batched_val_data, optimizer, criterion)
    val_losses.append(val_loss)

    increasing_val_losses_tail_len = 0
    for loss in val_losses[-IMPATIENCE_LIMIT:]:
        if loss > val_losses[-IMPATIENCE_LIMIT-1]:
            increasing_val_losses_tail_len += 1

    print(f"epoch {epoch_num}", 
          f"train loss {train_loss}",
          f"val loss {val_loss}",
          f"impatience {increasing_val_losses_tail_len} of {IMPATIENCE_LIMIT}",
          sep='\t')

    if increasing_val_losses_tail_len >= IMPATIENCE_LIMIT:
        print("impatience stop in epoch", epoch_num)
        break
'''

'\nmodel = LSTMModel(len(word2ix), len(tag2ix), emb_dim=100, hidden_dim=100)\n\ncriterion = torch.nn.CrossEntropyLoss(ignore_index=tag2ix[PAD_TAG])\nmodel = model.to(DEVICE)\noptimizer = torch.optim.Adam(params=model.parameters())\ncriterion = criterion.to(DEVICE)\n\n\nEPOCHS = 100\nIMPATIENCE_LIMIT = 5\n\nval_losses = [float("inf")] * (IMPATIENCE_LIMIT + 1)\nfor epoch_num in range(EPOCHS):\n    train_loss = train_epoch(model, padded_batched_train_data, optimizer, criterion)\n    val_loss = test_epoch(model, padded_batched_val_data, optimizer, criterion)\n    val_losses.append(val_loss)\n\n    increasing_val_losses_tail_len = 0\n    for loss in val_losses[-IMPATIENCE_LIMIT:]:\n        if loss > val_losses[-IMPATIENCE_LIMIT-1]:\n            increasing_val_losses_tail_len += 1\n\n    print(f"epoch {epoch_num}", \n          f"train loss {train_loss}",\n          f"val loss {val_loss}",\n          f"impatience {increasing_val_losses_tail_len} of {IMPATIENCE_LIMIT}",\n          sep=\'\t\')\

### оценка модели

Выберем случайное предложение из тестовой выборки, посмотрим на предсказание тегов для него моделью.

In [26]:
'''
import random

# давайте возьмём случайный батч данных
some_batch_src, some_batch_tgt = batched_train_data[random.randrange(0, len(batched_train_data))]
some_batch_src2tgt = list(zip(some_batch_src, some_batch_tgt))

# давайте возьмём случайный элемент из выбранного батча
some_src, some_tgt = some_batch_src2tgt[random.randrange(0, len(some_batch_src2tgt))]
print(some_src, some_tgt)


src_tensor = torch.LongTensor([some_src]).to(DEVICE)  # тривиальный батч из выбранного эл-та
tgt_tensor = torch.LongTensor([some_tgt]).to(DEVICE)  # тривиальный батч из выбранного эл-та
model_results, joint_scores, mean_scores = batch_score(src_tensor, tgt_tensor)

print(f"mean accuracy:\t{mean_scores[0]}", 
      f"joint accuracy:\t{joint_scores[0]}",
      '*' * 3,
      sep='\n')
import pandas as pd
pd.DataFrame(model_results[0], 
             columns=["token", "gold tag", "pred tag"])
'''

'\nimport random\n\n# давайте возьмём случайный батч данных\nsome_batch_src, some_batch_tgt = batched_train_data[random.randrange(0, len(batched_train_data))]\nsome_batch_src2tgt = list(zip(some_batch_src, some_batch_tgt))\n\n# давайте возьмём случайный элемент из выбранного батча\nsome_src, some_tgt = some_batch_src2tgt[random.randrange(0, len(some_batch_src2tgt))]\nprint(some_src, some_tgt)\n\n\nsrc_tensor = torch.LongTensor([some_src]).to(DEVICE)  # тривиальный батч из выбранного эл-та\ntgt_tensor = torch.LongTensor([some_tgt]).to(DEVICE)  # тривиальный батч из выбранного эл-та\nmodel_results, joint_scores, mean_scores = batch_score(src_tensor, tgt_tensor)\n\nprint(f"mean accuracy:\t{mean_scores[0]}", \n      f"joint accuracy:\t{joint_scores[0]}",\n      \'*\' * 3,\n      sep=\'\n\')\nimport pandas as pd\npd.DataFrame(model_results[0], \n             columns=["token", "gold tag", "pred tag"])\n'

Посчитаем наши `mean` и `join` метрики на всей тестовой выборке, перебрав все батчи. 

In [27]:
'''
joint_accuracies = []
mean_accuracies = []
for src, tgt in padded_batched_test_data:
    src_tensor = torch.LongTensor(src).to(DEVICE)
    tgt_tensor = torch.LongTensor(tgt).to(DEVICE)

    t2t2t, joints, means = batch_score(src_tensor, tgt_tensor)
    joint_accuracies.extend(joints)
    mean_accuracies.extend(means)
# Посмотрим среднее значение mean и join метрик на элементах тестовой выборки
{"joint": np.mean(joint_accuracies), 
 "mean": np.mean(mean_accuracies)}
'''

'\njoint_accuracies = []\nmean_accuracies = []\nfor src, tgt in padded_batched_test_data:\n    src_tensor = torch.LongTensor(src).to(DEVICE)\n    tgt_tensor = torch.LongTensor(tgt).to(DEVICE)\n\n    t2t2t, joints, means = batch_score(src_tensor, tgt_tensor)\n    joint_accuracies.extend(joints)\n    mean_accuracies.extend(means)\n# Посмотрим среднее значение mean и join метрик на элементах тестовой выборки\n{"joint": np.mean(joint_accuracies), \n "mean": np.mean(mean_accuracies)}\n'

Модель ошибается на примерно пятой части токенов, а верно теггированы целиком примерно пятая часть предложений.

# Секция для решения домашнего задания.

Выше в тетради представлены два бейзлайна для решения задачи классификации. Мы оценивали эти бейзлайны на отложенной выборке.
Мы (организаторы курса) также отложили ещё один кусок выборки и оценили бейзлайны на нём.

**Ваша задача** &mdash; написать нейросетевую модель, результат работы которой превзойдёт реализованные выше бейзлайны.

### Формат сдачи задания
Отправить на проверку ваше решение предстоит следующим образом: 
* нужно вашей моделью предсказать теги для проверочной выборки, 
* в форме для сдачи задания прикрепить два файла:
  1. файл с предсказанием вашей модели на проверочной выборке (назовите файл так, чтобы его расширение было `.bio`) 
  2. zip-архив с кодом, в котором описана ваша модель и то, как она обучается.

In [28]:
import pandas as pd

In [47]:
!gdown --id 16ZylRqsmwBIOtuLC9hqmIIhfHdGX7hI3

df_test = pd.read_csv('/content/assignment_test_sample_solution.tsv', sep='\t')
df_test.head(50)

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=16ZylRqsmwBIOtuLC9hqmIIhfHdGX7hI3
To: /content/assignment_test_sample_solution.tsv
100% 153k/153k [00:00<00:00, 72.0MB/s]


,PART,O
0,I.,B-GPE
1,FRIENDS,O
2,OF,O
3,CHILDHOOD,I-PER
4,I,O
5,Dr.,B-GPE
6,Howard,O
7,Archie,O
8,had,O
9,just,O


In [32]:
PAD_TOKEN = "PAD_TOKEN"
vocab_big = set([token for sentence_tokens in train_sents_tokens+test_sents_tokens
    for token in sentence_tokens] + [token for token in list(df_test.PART)] )
vocab_big.add(PAD_TOKEN)
vocab_big = list(vocab_big)


PAD_TAG = "PAD_TAG"
tagset = set([tag
            for sentence_tags in train_sents_tags+test_sents_tags
            for tag in sentence_tags]+ [token for token in list(df_test.O)])
tagset.add(PAD_TAG)
tagset = list(tagset)

print(f"vocab size: {str(len(vocab_big))}, tagset_size: {str(len(tagset))}")
print("vocab samples:", vocab_big[:5])
print("tagset samples:", tagset[:5])


ix2word_big = dict(enumerate(vocab_big))
word2ix_big = {w:ix for ix, w in ix2word_big.items()}

ix2tag_big = dict(enumerate(tagset))
tag2ix_big = {t:ix for ix, t in ix2tag_big.items()}

vocab size: 6647, tagset_size: 9
vocab samples: ['uncombed', 'superintending', 'if', 'breasts', 'indeterminate']
tagset samples: ['B-GPE', 'I-FAC', 'I-LOC', 'B-PER', 'O']


In [ ]:
def batch_score_test(src_tensor, tgt_tensor, model):
    """returns mean and joint scores on data batch"""
    src = src_tensor.tolist()
    tgt = tgt_tensor.tolist()

    pred, _ = model(src_tensor)  # получаем предсказания модели на элементах батча
    
    batch_tok2tag2pred = []
    batch_joint_accuracies = []
    batch_mean_accuracies = []
    for some_src, some_tgt, tags_predicted_digitized in zip(src, 
                                                            tgt, 
                                                            pred.argmax(dim=-1).tolist()):
        # восстанавливаем токены, предсказанные и ожидаемые метки токенов, из их численных записей
        tokens = list(map(ix2word_big.get, some_src))
        tags_true = list(map(ix2tag_big.get, some_tgt))
        tags_pred = list(map(ix2tag_big.get, tags_predicted_digitized))

        tok2tag2pred = list(zip(tokens, tags_true, tags_pred))
        
        batch_tok2tag2pred.append(tok2tag2pred)

        correct_preds = []  # чтобы хранить верные предсказания модели в человекочитаемом виде
        actual_preds = []  # чтобы хранить *все* предсказания модели в человекочитаемом виде
        for tok, gold_tag, pred_tag in tok2tag2pred:
            if gold_tag == PAD_TAG:
                # нас не волнуют ошибки на паддингах, тк они находятся за пределами предложения
                break

            batch_entry_w_prediction = (tok, gold_tag, pred_tag)
            if gold_tag == pred_tag:
                correct_preds.append(batch_entry_w_prediction)
            actual_preds.append(batch_entry_w_prediction)

        correct_preds_num = len(correct_preds)
        joint_accuracy = int(correct_preds_num == len(actual_preds))  # 1 если не было ошибок, иначе 0
        mean_accuracy = correct_preds_num/len(actual_preds) if actual_preds else 0  # доля верных ответов
        batch_joint_accuracies.append(joint_accuracy)
        batch_mean_accuracies.append(mean_accuracy)


    return batch_tok2tag2pred, batch_joint_accuracies, batch_mean_accuracies



# кодировать решение задания можно, например, в этой ячейке
# например, ниже мы реализовали какую-то модель. 
# но вряд ли она сработеает лучше бейзлайна :)

class LSTM_sigmoid(torch.nn.Module):
    def __init__(self, 
                 input_dim, output_dim,
                 emb_dim, hidden_dim):
        super().__init__()

        self.emb = torch.nn.Embedding(input_dim, emb_dim)
        self.lstm = torch.nn.LSTM(emb_dim, hidden_dim,
                            num_layers=2,  bidirectional=True,dropout = 0.2)
        self.Dropout = torch.nn.Dropout(0.2)
      #  self.fc = torch.nn.Linear(hidden_dim, 2*hidden_dim)
        self.dc = torch.nn.Linear(2*hidden_dim,output_dim)
    
    def forward(self, in_seqs_batch):
        embedded = self.emb(in_seqs_batch)
        drop = self.Dropout(embedded)
        rnn_out, hc = self.lstm(embedded)
        #lin = self.fc(rnn_out)
        out = self.dc(rnn_out)
        return out, hc

In [ ]:
# кодировать решение задания можно, например, в этой ячейке
# например, ниже мы реализовали какую-то модель. 
# но вряд ли она сработеает лучше бейзлайна :)

class LSTM_sigmoid(torch.nn.Module):
    def __init__(self, 
                 input_dim, output_dim,
                 emb_dim, hidden_dim,num_layers,dropoutl,dropoutle):
        super().__init__()

        self.emb = torch.nn.Embedding(input_dim, emb_dim)
        self.lstm = torch.nn.LSTM(emb_dim, hidden_dim,
                            num_layers=num_layers,  bidirectional=True,dropout = dropoutl)
        self.Dropout = torch.nn.Dropout(dropoutle)
      #  self.fc = torch.nn.Linear(hidden_dim, 2*hidden_dim)
        self.dc = torch.nn.Linear(2*hidden_dim,output_dim)
    
    def forward(self, in_seqs_batch):
        embedded = self.emb(in_seqs_batch)
        drop = self.Dropout(embedded)
        rnn_out, hc = self.lstm(embedded)
        #lin = self.fc(rnn_out)
        out = self.dc(rnn_out)
        return out, hc

Секция для обучения описанной вами модели

In [ ]:
# в этой ячейке можно написать код, 
emb_dim = 8000
hidden_dim = 1024
num_layers = 2
dropoutl = 0.5
dropoutle = 0.2

EPOCHS = 15
IMPATIENCE_LIMIT = 20

# который будет обучать описанную вами модель
model = LSTM_sigmoid(len(word2ix_big), len(tag2ix_big), emb_dim, hidden_dim,num_layers,dropoutl,dropoutle)

criterion = torch.nn.CrossEntropyLoss(ignore_index=tag2ix_big[PAD_TAG])
model = model.to(DEVICE)
optimizer = torch.optim.Adam(params=model.parameters())
criterion = criterion.to(DEVICE)


val_losses = [float("inf")] * (IMPATIENCE_LIMIT + 1)
for epoch_num in range(EPOCHS):
    train_loss = train_epoch(model, padded_batched_train_data, optimizer, criterion)
    val_loss = test_epoch(model, padded_batched_val_data, optimizer, criterion)
    val_losses.append(val_loss)

    increasing_val_losses_tail_len = 0
    for loss in val_losses[-IMPATIENCE_LIMIT:]:
        if loss > val_losses[-IMPATIENCE_LIMIT-1]:
            increasing_val_losses_tail_len += 1

    print(f"epoch {epoch_num}", 
          f"train loss {train_loss}",
          f"val loss {val_loss}",
          f"impatience {increasing_val_losses_tail_len} of {IMPATIENCE_LIMIT}",
          sep='\t')

    if increasing_val_losses_tail_len >= IMPATIENCE_LIMIT:
        print("impatience stop in epoch", epoch_num)
        break

Секция для вычисления предсказаний модели на отложенных данных.

In [ ]:
model_results_test =[]
joint_accuracies=[]
mean_accuracies=[]
for src, tgt in padded_batched_test_data:
    src_tensor = torch.LongTensor(src).to(DEVICE)
    tgt_tensor = torch.LongTensor(tgt).to(DEVICE)

    results_test, joints, means  = batch_score_test(src_tensor, tgt_tensor, model)

    joint_accuracies.extend(joints)
    mean_accuracies.extend(means)
    model_results_test.extend(results_test)



In [ ]:
model_results_test

In [ ]:
{"joint": np.mean(joint_accuracies), 
 "mean": np.mean(mean_accuracies)}

In [ ]:
{'joint': 0.7048192771084337, 'mean': 0.9419308451190243}

In [ ]:
# в этой ячейке можно написать код, 
# который будет использовать обученную модель
# для разметки проверочного датасета.


In [ ]:
df_test['token'] = df_test['PART'].apply(lambda x: word2ix_big[x])
df_test['tag'] = df_test['O'].apply(lambda x: tag2ix_big[x])

In [ ]:
df_test.head(10)

In [80]:
# первый вариант разбиения просто по 4 символа
'''
df_test_token_oll =[]
df_test_tag_oll =[]
for j in range(len(df_test)//4 + (0 if len(df_test)%4==0 else 1)):
  i = j*4
  df_test_token_oll.append([item for item in df_test.token[i:i+4]])
  df_test_tag_oll.append([item for item in df_test.tag[i:i+4]])
'''
# второй вариант разбиения - по знакам препинания 
df_test_token,df_test_token_oll =[],[]
df_test_tag,df_test_tag_oll =[], []

for i, row in enumerate(df_test.PART):
  #print(i, row, len(row) )
  if len(row) > 12:
    df_test_token.append(word2ix_big['I'])
    df_test_tag.append(tag2ix_big['O'])
  if row !='.' and row !='!' and row !='?' :
    df_test_token.append(word2ix_big[row])
    df_test_tag.append(tag2ix_big[df_test.O[i]])

  else:

    df_test_token.append(word2ix_big[row])
    df_test_tag.append(tag2ix_big['O'])

    df_test_token_oll.append(df_test_token)
    df_test_tag_oll.append(df_test_tag)

    df_test_token =[]
    df_test_tag =[]


test_data = list(zip(df_test_token_oll, df_test_tag_oll))

batched_dataset_test = generate_batched_dataset(test_data, BATCH_SIZE)

In [81]:
padded_batched_test_data_oll = pad_batched_dataset_of_sequences_pairs(batched_dataset_test, 
                                                                  word2ix_big[PAD_TOKEN],
                                                                  tag2ix_big[PAD_TAG]) 

In [ ]:
joint_accuracies = []
mean_accuracies = []
t2t2t_accuracies = []
for src, tgt in padded_batched_test_data_oll:
    src_tensor = torch.LongTensor(src).to(DEVICE)
    tgt_tensor = torch.LongTensor(tgt).to(DEVICE)

    t2t2t, joints, means = batch_score_test(src_tensor, tgt_tensor,model)
    joint_accuracies.extend(joints)
    mean_accuracies.extend(means)
    t2t2t_accuracies.extend(t2t2t)

In [ ]:
sample_submission = pd.DataFrame([i for sm in t2t2t_accuracies for i in sm], 
             columns=["PART", "gold", "O"])[["PART", "O"]]

sample_submission

In [ ]:
sample_submission.O.value_counts()

In [ ]:
sample_submission.to_csv('sample_submission7.bio', index=False)

В описании задания на онлайн-платформе, в файле `...sample_solution.tsv` приложен пример файла с предсказаниями модели для сдачи задания.